In [ ]:
import sys
sys.path.insert(0, "../segmentation_models")
sys.path.insert(0,"../classification_models")

import collections
import cv2 
import gc
import glob
import json
import keras
import numpy as np
import os
import classification_models
import segmentation_models
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy import io
from sklearn.preprocessing import normalize
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Dense
from keras.optimizers import Adam
from segmentation_models import Unet
from segmentation_models.utils import set_trainable
from tqdm import tqdm_notebook as tqdm


In [ ]:
input_size = (512,512)

def load_image(path):
    """Load grayscale image from path"""
    return cv2.resize(cv2.imread(path,1), input_size)

def load_binary_image(path):
    """Load grayscale image from path"""
    return cv2.resize(cv2.imread(path,0), input_size)

def load_mat(path):
    """Load grayscale image from path"""
    image = io.loadmat(path, appendmat=False)['dxImage']['img'][0][0]
    normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))
    colored_image = cv2.cvtColor(normalized_image, cv2.COLOR_GRAY2RGB)
    return cv2.resize(colored_image, input_size)

def load_mask_mat(path):
    """Load grayscale image from path"""
    # import matlab files, extracting array of masks
    mask_list = np.stack(io.loadmat(path, appendmat=False)['maskImage']['maskCrop'][0][0][0], axis=0)
    line_num = len(mask_list)
    
    # create placeholder arrays
    foreground = np.zeros((input_size[0], input_size[1], 1))
    background = np.ones((input_size[0], input_size[1], 1))
    
    # for each mask, scale it, reshape it, and add it to the foreground
    for i, mask in enumerate(mask_list):
        mask_array = cv2.resize(mask.astype(np.uint8), input_size)
        scaled_mask_array = np.reshape(mask_array, (input_size[0], input_size[1], 1))
        foreground = np.logical_or(foreground, scaled_mask_array)
    foreground = np.reshape(foreground, (1, input_size[0], input_size[1], 1))
    
    # create the background mask
    background = 1 - foreground
    
    # combine the background and foreground masks into a single array
    final_mask_list = np.array(np.append(background, foreground, axis=3))
    return final_mask_list

X = []
Y = []
data_dir = "/data/midi-lab-general/osemis_annotations/osemis_annotation_file_to_masks/Masks/*.mat"
for filepath in tqdm(sorted(glob.glob(data_dir, recursive=True))):
    if filepath.endswith("image.mat"):
        X.append(load_mat(filepath))
    if filepath.endswith("mask.mat"):
        Y.append(load_mask_mat(filepath))
X = np.reshape(np.array(X), (-1, input_size[0], input_size[1], 3))
Y = np.reshape(np.array(Y), (len(X), input_size[0], input_size[1], 2))

line_ratio = 0
for image in Y[:,:,:,0]:
    image = np.round(image, decimals=0)
    unique, counts = np.unique(image, return_counts=True)
    d = dict(zip(unique, counts))
    line_ratio += d[1.0]/(d[1.0] + d[0.0])
print(line_ratio/len(Y))

In [ ]:
val_split = 0.15
test_split = 0.15
n = len(X)
sp1 = int(((1 - val_split - test_split) * n) - 0.5) # Choose first index with rounding adjustment (0.5)
sp2 = int(((1 - test_split) * n) - 0.5) # Choose second index with rounding adjustment (0.5)
X_train, Y_train = X[:sp1], Y[:sp1]
X_val, Y_val = X[sp1:sp2], Y[sp1:sp2]
X_test, Y_test = X[sp2:], Y[sp2:]
X_train = X_train[:15]
Y_train = Y_train[:15]
print("{} Samples; {} train; {} val; {} test".format(n, len(X_train), len(X_val), len(X_test)))
def plot_sample(sample, figsize=(10,10)):
    print(sample.shape)
    sample = np.squeeze(sample)
    plt.figure(figsize=figsize)
    plt.imshow(sample, cmap='gray')
    plt.show()
def plot_mask(sample, mask, figsize=(10,10)):
    mask = np.squeeze(mask)
    plt.figure(figsize=figsize)
    plt.imshow(sample, cmap='gray')
    plt.imshow(np.ma.masked_where(mask[:, :, 1] <= 0.5, mask[:, :, 1]), cmap='jet', alpha=0.5, vmin=0, vmax=1)
    plt.show()
plot_mask(X[2], Y[2])

In [ ]:
epsilon = 0.0001

def dice_coef(y_true, y_pred):
    y_true = y_true[:, :, :, 1]
    y_pred = y_pred[:, :, :, 1]
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + epsilon) / (K.sum(y_true_f) + K.sum(y_pred_f) + epsilon)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def weighted_cce(y_true, y_pred):
    p = K.sum(K.flatten(y_true[...,:1])) / (224**2)
    maximum_w = 1 / K.log(1.02 + 0)
    minimum_w = 1 / K.log(1.02 + 1)
    w = 1 / K.log(1.02 + p)
    scaled_w = (w - minimum_w) / (maximum_w - minimum_w)
    
    y_pred = K.clip(y_pred, 1e-7, 1 - 1e-7)
    loss_map = -K.sum(scaled_w * y_true * K.log(y_pred), axis=-1)
    return K.mean(K.flatten(loss_map))

def weighted_cce_plus_dice_coef_loss(y_true, y_pred):
    return 0.5*weighted_cce(y_true, y_pred) + 0.5*dice_coef_loss(y_true, y_pred)

def unbalanced_weighted_cce_plus_dice_coef_loss(y_true, y_pred):
    return 0.1*weighted_cce(y_true, y_pred) + 0.9*dice_coef_loss(y_true, y_pred)

In [ ]:
# Create UNet model with custom resnet weights
model = Unet('resnet50_custom', 
             classes=2, 
             input_shape=(512, 512, 3), 
             encoder_weights='imagenet', 
             activation='softmax')


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss=dice_coef_loss, metrics=["accuracy", dice_coef])

In [ ]:
# Choose checkpoint path
checkpoint_path = "./UNET-with-New-ResNet50_weights"
os.makedirs(checkpoint_path, exist_ok=True)

# Create early stopping callback
early_stopping = EarlyStopping(monitor="val_dice_coef", mode="max", patience=75, verbose=1)

# Create checkpointer to save best model weights
checkpointer = ModelCheckpoint(filepath=checkpoint_path + "/resnet50-custom_small_dice_weights.h5", verbose=1, monitor="val_dice_coef", mode="max", save_best_only=True)

# TEST: ReduceLROnPlateau
callback_list = [checkpointer, early_stopping]

In [ ]:
res_history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=8, epochs=50, verbose=1, callbacks=callback_list)

In [ ]:
# Save history
history_path = checkpoint_path + "/History/" + "resnet50-custom_small_dice_history.json"
print("Writing history dictionary to {}".format(history_path))
with open(history_path, 'w') as history_file:
    json.dump(res_history.history, history_file)

In [ ]:
model.load_weights(checkpoint_path + "/resnet50-custom_small_dice_weights.h5")
loss, accuracy, dice = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
print('Dice coef:', dice)

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(14,10))

plt.plot(res_history.history['acc'])
plt.plot(res_history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(res_history.history['loss'])
plt.plot(res_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation dice values
plt.plot(res_history.history['dice_coef'])
plt.plot(res_history.history['val_dice_coef'])
plt.title('Model dice coef')
plt.ylabel('Dice coef')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
test_id = 1
predicted_mask = model.predict(np.expand_dims(X_test[test_id], axis=0))
plot_mask(X_test[test_id], predicted_mask, figsize=(10,10))
plot_mask(X_test[test_id], Y_test[test_id], figsize=(10,10))

In [ ]:
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.Session(config=config))
reset_keras()

In [ ]:
# Create UNet model with imagenet weights
model = Unet('resnet50', 
             classes=2, 
             input_shape=(512, 512, 3), 
             encoder_weights='imagenet', 
             activation='softmax')

In [ ]:
# Choose checkpoint path
checkpoint_path = "./UNET-with-New-ResNet50_weights"
os.makedirs(checkpoint_path, exist_ok=True)

# Create early stopping callback
early_stopping = EarlyStopping(monitor="val_dice_coef", mode="max", patience=100)

# Create checkpointer to save best model weights
checkpointer = ModelCheckpoint(filepath=checkpoint_path + "/resnet50_small_dice_weights.h5", verbose=1, monitor="val_dice_coef", mode="max", save_best_only=True)

# TEST: ReduceLROnPlateau
callback_list = [checkpointer, early_stopping]

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss=dice_coef_loss, metrics=["accuracy", dice_coef])

In [ ]:
imagenet_history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=8, epochs=50, verbose=1, callbacks=callback_list)

In [ ]:
model.load_weights(checkpoint_path + "/resnet50_small_dice_weights.h5")
loss, accuracy, dice = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
print('Dice coef:', dice)

In [ ]:
custom_history = json.load(open(history_path, "r"))

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(14,5))
plt.plot(imagenet_history.history['acc'])
plt.plot(imagenet_history.history['val_acc'])
plt.plot(custom_history['acc'])
plt.plot(custom_history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['ImageNet Train', 'ImageNet Val', 'Custom Train', 'Custom Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(14,5))
plt.plot(imagenet_history.history['loss'])
plt.plot(imagenet_history.history['val_loss'])
plt.plot(custom_history['loss'])
plt.plot(custom_history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['ImageNet Train', 'ImageNet Val', 'Custom Train', 'Custom Val'], loc='upper left')
plt.show()

# Plot training & validation dice values
plt.figure(figsize=(14,5))
plt.plot(imagenet_history.history['dice_coef'])
plt.plot(imagenet_history.history['val_dice_coef'])
plt.plot(custom_history['dice_coef'])
plt.plot(custom_history['val_dice_coef'])
plt.title('Model dice coef')
plt.ylabel('Dice coef')
plt.xlabel('Epoch')
plt.legend(['ImageNet Train', 'ImageNet Val', 'Custom Train', 'Custom Val'], loc='upper left')
plt.show()

In [ ]:
test_id = 1
predicted_mask = model.predict(np.expand_dims(X_test[test_id], axis=0))
plot_mask(X_test[test_id], predicted_mask)
plot_mask(X_test[test_id], Y_test[test_id])